In [1]:
require 'nn'
require 'optim'

torch.manualSeed(0)
torch.setnumthreads(4)

In [2]:
classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 }
geometry = { 32, 32 }

net = nn.Sequential()

net:add(nn.SpatialConvolution(3, 6, 5, 5))
net:add(nn.ReLU())
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.SpatialConvolution(6, 16, 5, 5))
net:add(nn.ReLU())
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.View(16*5*5))
net:add(nn.Linear(16*5*5, 120))
net:add(nn.ReLU())
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())
net:add(nn.Linear(84, #classes))
net:add(nn.LogSoftMax())

parameters, gradParameters = net:getParameters()
criterion = nn.ClassNLLCriterion()

In [3]:
train = torch.load('mnist-p2b-train.t7')
train_amount = train.data:size()[1]

test = torch.load('mnist-p2b-test.t7')
test_amount = test.data:size()[1]

In [4]:
train_classes = torch.Tensor(train_amount, #classes)
test_classes = torch.Tensor(test_amount, #classes)

for i = 1, train_amount do
    vector = torch.zeros(#classes)
    vector[train.label[i]] = 1
    train_classes[i] = vector
end

for i = 1, test_amount do
    vector = torch.zeros(#classes)
    vector[test.label[i]] = 1
    test_classes[i] = vector
end

train.data = train.data:double()
test.data = test.data:double()

In [5]:
mean = {}
stdev = {}

for i = 1, 3 do
    mean[i] = train.data[{ {}, {i}, {}, {} }]:mean()
    train.data[{ {}, {i}, {}, {} }]:add(-mean[i])
    
    stdev[i] = train.data[{ {}, {i}, {}, {} }]:std()
    train.data[{ {}, {i}, {}, {} }]:div(stdev[i])
    print('Train Channel ' .. i .. ' Mean: ', mean[i], ' Stdev: ', stdev[i])
end


for i = 1, 3 do
    mean[i] = test.data[{ {}, {i}, {}, {} }]:mean()
    test.data[{ {}, {i}, {}, {} }]:add(-mean[i])
    
    stdev[i] = test.data[{ {}, {i}, {}, {} }]:std()
    test.data[{ {}, {i}, {}, {} }]:div(stdev[i])
    print('Test Channel ' .. i .. ' Mean: ', mean[i], ' Stdev: ', stdev[i])
end

Train Channel 1 Mean: 	113.22781756185	 Stdev: 	67.791216435084	


Train Channel 2 Mean: 	107.1339985026	 Stdev: 	64.19881938213	


Train Channel 3 Mean: 	93.089492203776	 Stdev: 	69.49323957483	


Test Channel 1 Mean: 	112.923084375	 Stdev: 	67.446258384046	


Test Channel 2 Mean: 	106.68813339844	 Stdev: 	63.735822418258	


Test Channel 3 Mean: 	92.559736328125	 Stdev: 	69.015380306025	


In [7]:
confusion = optim.ConfusionMatrix(classes)
epoch_limit = 128

In [8]:
batch_size = 10
for epoch = 1, epoch_limit do
    confusion:zero()
    
    for t = 1, train_amount, batch_size do
        local inputs = torch.Tensor(batch_size, 3, geometry[1], geometry[2])
        local targets = torch.Tensor(batch_size)
        local k = 1
        
        for i = t, math.min(t + batch_size - 1, train_amount) do
            local input = train.data[i]:clone()
            local _, target = train_classes[i]:clone():max(1)
            target = target:squeeze()
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local feval = function(x)
            collectgarbage()

            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()

            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)

            for i = 1, batch_size do
                confusion:add(outputs[i], targets[i])
            end

            return f, gradParameters
        end
        
        sgd_state = sgd_state or {
            learning_rate = 0.06,
            momentum = 0,
            learning_rate_decay = 5e-7
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    print('Epoch: ' .. epoch .. ' | Average valid: ' .. confusion.averageValid * 100 .. '%')
    print(confusion)
end

print(confusion)

Epoch: 1 | Average valid: 0%	
ConfusionMatrix:
[[       0     472     419     369       0     126       0    1064    2909       0       0]   0.000% 	[class: 1]
 [       0     783     253     369       0      82       0    1268    3383       0       0]   12.757% 	[class: 2]
 [       0     584     402     413       0      96       0    1111    2811       0       0]   7.421% 	[class: 3]
 [       0     487     295     414       0      91       0    1232    3075       0       0]   7.401% 	[class: 4]
 [       0     545     386     420       0     164       0    1275    2541       0       0]   0.000% 	[class: 5]
 [       0     479     322     340       0      89       0    1044    2646       0       0]   1.809% 	[class: 6]
 [       0     518     339     368       0      89       0    1202    2869       0       0]   0.000% 	[class: 7]
 [       0     518     308     434       0     139       0    1480    2779       0       0]   26.158% 	[class: 8]
 [       0     523     344     382       0     

  nclasses : 11
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 11
  totalValid : 0.09955
}


Epoch: 2 | Average valid: 0%	
ConfusionMatrix:
[[       0    4878      40       6       0       0       0     432       3       0       0]   0.000% 	[class: 1]
 [       0    5703       1       5       0       0       0     425       4       0       0]   92.913% 	[class: 2]
 [       0    4958      20      11       0       0       0     424       4       0       0]   0.369% 	[class: 3]
 [       0    5085      18       4       0       0       0     483       4       0       0]   0.072% 	[class: 4]
 [       0    4837      26       4       0       0       0     464       0       0       0]   0.000% 	[class: 5]
 [       0    4515      11       6       0       0       0     386       2       0       0]   0.000% 	[class: 6]
 [       0    4888      18       4       0       0       0     474       1       0       0]   0.000% 	[class: 7]
 [       0    5050      10      10       0       0       0     587       1       0       0]   10.375% 	[class: 8]
 [       0    4919      20       7       0     

Epoch: 3 | Average valid: 0%	


ConfusionMatrix:
[[      21    5037     286       2       0       0       0      13       0       0       0]   0.392% 	[class: 1]
 [       0    6121      11       2       0       0       0       4       0       0       0]   99.723% 	[class: 2]
 [       1    5306     102       0       0       0       0       8       0       0       0]   1.883% 	[class: 3]
 [       1    5526      51       1       0       0       0      15       0       0       0]   0.018% 	[class: 4]
 [       0    5229      90       1       0       0       0      11       0       0       0]   0.000% 	[class: 5]
 [       1    4849      60       1       0       0       0       9       0       0       0]   0.000% 	[class: 6]
 [       2    5276     100       0       0       0       0       7       0       0       0]   0.000% 	[class: 7]
 [       1    5565      61       1       0       0       0      30       0       0       0]   0.530% 	[class: 8]
 [       0    5276      46       1       0       0       0       6       0    

...

Epoch: 127 | Average valid: 0%	


ConfusionMatrix:
[[    5350       1       0       0       0       0       1       0       1       1       5]   99.832% 	[class: 1]
 [       1    6127       1       1       1       0       1       0       0       0       6]   99.821% 	[class: 2]
 [       1       0    5403       1       0       0       0       2       0       1       9]   99.742% 	[class: 3]
 [       0       1       0    5585       0       1       0       0       1       0       6]   99.839% 	[class: 4]
 [       1       3       0       0    5316       0       2       1       1       2       5]   99.719% 	[class: 5]
 [       0       0       0       1       0    4914       0       0       0       1       4]   99.878% 	[class: 6]
 [       1       0       0       0       1       0    5374       1       0       0       8]   99.796% 	[class: 7]
 [       1       0       0       0       1       0       0    5648       0       3       5]   99.823% 	[class: 8]
 [       2       0       0       3       0       0       1       0    5

Epoch: 128 | Average valid: 0%	


ConfusionMatrix:
[[    5350       1       0       0       0       0       1       0       1       1       5]   99.832% 	[class: 1]
 [       0    6128       1       1       2       0       1       0       0       0       5]   99.837% 	[class: 2]
 [       2       0    5403       0       0       0       0       3       0       0       9]   99.742% 	[class: 3]
 [       0       0       1    5585       1       1       0       0       1       0       5]   99.839% 	[class: 4]
 [       2       1       0       0    5317       0       0       1       0       3       7]   99.737% 	[class: 5]
 [       0       0       0       1       0    4913       0       0       0       1       5]   99.858% 	[class: 6]
 [       1       0       0       0       1       0    5374       1       0       0       8]   99.796% 	[class: 7]
 [       1       1       0       1       1       0       0    5646       0       3       5]   99.788% 	[class: 8]
 [       2       0       0       3       0       0       1       0    5

      10 : 10
      11 : 11
    }
  _prediction : FloatTensor - size: 11
  _pred_idx : LongTensor - size: 1
  nclasses : 11
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 11
  totalValid : 0.99788333333333
}


In [9]:
batch_size = 10
confusion:zero()
for t = 1, test_amount, batch_size do
    local inputs = torch.Tensor(batch_size, 3, geometry[1], geometry[2])
    local targets = torch.Tensor(batch_size)
    local k = 1
    
    for i = t, math.min(t + batch_size - 1, test_amount) do
        local input = test.data[i]:clone()
        local _, target = test_classes[i]:clone():max(1)
        
        inputs[k] = input
        targets[k] = target
        k = k + 1
    end
    
    local preds = net:forward(inputs)
    
    for i = 1, batch_size do
        confusion:add(preds[i], targets[i])
    end
end

print(confusion)

ConfusionMatrix:
[[     778       7       9       9       6      28      27       6       7      11       4]   87.220% 	[class: 1]
 [       2     866      21      18      19      15      16      15      15       7      27]   84.819% 	[class: 2]
 [       8      17     755      38      13       9       4      35      20      11      22]   81.009% 	[class: 3]
 [      12      10      12     776      11      41       6       3      23      12       7]   84.995% 	[class: 4]
 [       8      13      10      11     716      13      16      15       8      53      21]   80.995% 	[class: 5]
 [      20       8       4      28      14     658      22       4      19      13      11]   82.147% 	[class: 6]
 [      29       8       9      17      27      22     714       4       9       6      17]   82.831% 	[class: 7]
 [       8      19      22      18      41       8       3     759       8      39      15]   80.745% 	[class: 8]
 [       6       9      22      40      24      29      17      13     

  _mat_flat : LongTensor - size: 121
  valids : FloatTensor - size: 11
  mat : LongTensor - size: 11x11
  averageUnionValid : 0.69778283075853
  _targ_idx : LongTensor - empty
  averageValid : 0.82157986814326
  classes : 
    {
      1 : 1
      2 : 2
      3 : 3
      4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
      11 : 11
    }
  _prediction : FloatTensor - size: 11
  _pred_idx : LongTensor - size: 1
  nclasses : 11
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 11
  totalValid : 0.8219
}
